In [1]:
import openpyxl
import os   

import pandas as pd
from deepdiff import DeepDiff
import json

from src.python import utils as utils
from src.python import functions as functions 

queries = utils.get_queries()
conn_replication = utils.get_connection_replication()
conn_local_env_replication = utils.get_connection_local_env_replication()
print(conn_replication.dsn)
print(conn_local_env_replication.dsn)
print(queries)


OperationalError: connection to server at "localhost" (::1), port 3435 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 3435 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [5]:
analysis_result = functions.analyze_schema_hierarchy(conn_replication, conn_local_env_replication, queries, debug=False)
functions.print_hierarchical_differences(analysis_result)
cleaned_results = functions.clean_hierarchical_differences(analysis_result)
print(cleaned_results)
functions.summarize_critical_differences(cleaned_results)

🔍 Building hierarchical schema structures...
Building schema structure for production...
Error executing tables: 'Queries' object has no attribute 'tables'
Error executing columns: 'Queries' object has no attribute 'columns'
Building schema structure for local...
Error executing tables: 'Queries' object has no attribute 'tables'
Error executing columns: 'Queries' object has no attribute 'columns'
🔍 Comparing nested structures...

HIERARCHICAL SCHEMA ANALYSIS
📊 Schemas compared: 0
📊 Total differences: 0

📊 CLEANED ANALYSIS:
  • Critical differences: 0
  • Important differences: 0
  • Noise (filtered): 0

🚨 ALL CRITICAL DIFFERENCES (0):

⚠️  IMPORTANT DIFFERENCES (0):
{'critical': [], 'important': [], 'noise': []}

DBA SUMMARY - CRITICAL SCHEMA DIFFERENCES

💡 RECOMMENDATIONS:
   2. Review table ownership differences between environments
   3. Examine data type changes for compatibility issues
   4. Create migrations to synchronize environments
   5. Establish process to prevent manual pr

In [3]:
# # Method 2: Execute all available queries
database_structure_source = {}
database_structure_target = {}
functions.execute_comparison_queries(conn_replication, queries, database_structure_source)
functions.execute_comparison_queries(conn_local_env_replication, queries, database_structure_target)

Executing 10 comparison queries: ['a_01_database', 'a_02_version', 'a_03_settings', 'a_05_event_triggers', 'a_06_extensions', 'a_10_languages', 'a_11_publications', 'a_12_schemas', 'b_09_functions', 'b_13_sequences']
Executing query: a_01_database
Executing query: a_02_version
Executing query: a_03_settings
Executing query: a_05_event_triggers
Executing query: a_06_extensions
Executing query: a_10_languages
Executing query: a_11_publications
Executing query: a_12_schemas
Executing query: b_09_functions
Executing query: b_13_sequences
Executing 10 comparison queries: ['a_01_database', 'a_02_version', 'a_03_settings', 'a_05_event_triggers', 'a_06_extensions', 'a_10_languages', 'a_11_publications', 'a_12_schemas', 'b_09_functions', 'b_13_sequences']
Executing query: a_01_database
Executing query: a_02_version
Executing query: a_03_settings
Executing query: a_05_event_triggers
Executing query: a_06_extensions
Executing query: a_10_languages
Executing query: a_11_publications
Executing quer

{'database': {'name': 'gigabeam_timescaledb',
  'properties': {'oid': '16384',
   'datname': 'gigabeam_timescaledb',
   'datdba': '10',
   'encoding': 6,
   'datlocprovider': 'c',
   'datistemplate': False,
   'datallowconn': True,
   'datconnlimit': -1,
   'datfrozenxid': '723',
   'datminmxid': '1',
   'dattablespace': '1663',
   'datcollate': 'en_US.utf8',
   'datctype': 'en_US.utf8',
   'daticulocale': None,
   'daticurules': None,
   'datcollversion': None,
   'datacl': None},
  'version': '16.3',
  'settings': {'DateStyle': {'name': 'DateStyle',
    'setting': 'ISO, MDY',
    'unit': None,
    'category': 'Client Connection Defaults / Locale and Formatting',
    'short_desc': 'Sets the display format for date and time values.',
    'extra_desc': 'Also controls interpretation of ambiguous date inputs.',
    'context': 'user',
    'vartype': 'string',
    'source': 'default',
    'min_val': None,
    'max_val': None,
    'enumvals': None,
    'boot_val': 'ISO, MDY',
    'reset_val'

In [4]:
functions.compare_database_structures(database_structure_source, database_structure_target,ignore_keys=['created_at', 'updated_at', 'oid'])

{'source_only': [],
 'target_only': ['database.settings.dynamic_library_path',
  'database.settings.hba_file',
  'database.settings.timescaledb.enable_hypertable_create',
  'database.settings.external_pid_file',
  'database.settings.ssl_max_protocol_version',
  'database.settings.timescaledb.enable_cagg_create',
  'database.settings.timescaledb.enable_hypertable_compression',
  'database.settings.log_directory',
  'database.settings.session_preload_libraries',
  'database.settings.ssl_dh_params_file',
  'database.settings.ssl_ciphers',
  'database.settings.data_directory',
  'database.settings.primary_conninfo',
  'database.settings.config_file',
  'database.settings.log_filename',
  'database.settings.ssl_ecdh_curve',
  'database.settings.shared_preload_libraries',
  'database.settings.unix_socket_directories',
  'database.settings.ssl_passphrase_command',
  'database.settings.jit_provider',
  'database.settings.krb_server_keyfile',
  'database.settings.ssl_min_protocol_version',
  'd

In [6]:
from simple_postgres_setup import setup_database, drop_database

config_path = '../setup/config.yml'

drop_database(config_path)

Access granted. Database will be dropped.


OperationalError: (psycopg2.errors.ObjectInUse) database "gigabeam_timescaledb" is being used by logical replication subscription
DETAIL:  There are 4 subscriptions.

[SQL: DROP DATABASE gigabeam_timescaledb]
(Background on this error at: https://sqlalche.me/e/20/e3q8)